# Model Loading Example
This notebook provides a experimental code of how to save and load tensorflow models. 

In [1]:
# Load Data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)
import tensorflow as tf

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


## Build a simple model
Our first step is to build a simple model for which we want to store.

In [2]:
# Define layers
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

# build inference graph
x = tf.placeholder(tf.float32, shape=[None, 784], name="input")
y_ = tf.placeholder(tf.float32, shape=[None, 10], name="prediction")

W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))

W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])
x_image = tf.reshape(x, [-1,28,28,1])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])
y_conv = tf.nn.softmax(tf.matmul(h_fc1, W_fc2) + b_fc2, name="output")

# build training graph
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_conv), reduction_indices=[1]))
train_step = tf.train.GradientDescentOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# run training
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

# To make this run faster, we'll only run 1,000 iterations of the training process.
for i in range(1):
    batch = mnist.train.next_batch(50)
    predictions, loss = sess.run([y_conv, accuracy], feed_dict={x: batch[0], y_: batch[1]})
    print "loss for iteration {} is {}".format(i, loss)    

loss for iteration 0 is 0.140000000596


# Save Model

In [3]:
# Create prediction signature
tensor_info_x = tf.saved_model.utils.build_tensor_info(x)
tensor_info_y = tf.saved_model.utils.build_tensor_info(y_)

prediction_signature = tf.saved_model.signature_def_utils.build_signature_def(
          inputs={'x': tensor_info_x},
          outputs={'y': tensor_info_y},
          method_name="prediction_signature")

# Saving
export_dir = "/tmp/save_model_directory_test4"
builder = tf.saved_model.builder.SavedModelBuilder(export_dir)
builder.add_meta_graph_and_variables(sess,
                                     ["tag"], 
                                     signature_def_map={"predict": prediction_signature})
builder.save()

INFO:tensorflow:No assets to save.


INFO:tensorflow:No assets to save.


INFO:tensorflow:No assets to write.


INFO:tensorflow:No assets to write.


INFO:tensorflow:SavedModel written to: /tmp/save_model_directory_test4/saved_model.pb


INFO:tensorflow:SavedModel written to: /tmp/save_model_directory_test4/saved_model.pb


'/tmp/save_model_directory_test4/saved_model.pb'

# Load Model

In [4]:
with tf.Session(graph=tf.Graph()) as sess1:
    tf.saved_model.loader.load(sess1, ["tag"], export_dir)
    graph = tf.get_default_graph()
    x = graph.get_tensor_by_name("input:0")
    y = graph.get_tensor_by_name("output:0")
    
    y_hat = sess1.run(fetches=y, feed_dict={x: batch[0]})

INFO:tensorflow:Restoring parameters from /tmp/save_model_directory_test4/variables/variables


INFO:tensorflow:Restoring parameters from /tmp/save_model_directory_test4/variables/variables


In [10]:
predictions[0]

array([  7.70880608e-04,   2.91431410e-04,   2.49422248e-02,
         2.08109291e-03,   9.95373204e-02,   3.54229063e-02,
         2.72273435e-04,   1.69640480e-04,   1.24033010e-02,
         8.24108899e-01], dtype=float32)

In [9]:
y_hat[0]

array([  7.70880608e-04,   2.91431410e-04,   2.49422248e-02,
         2.08109291e-03,   9.95373204e-02,   3.54229063e-02,
         2.72273435e-04,   1.69640480e-04,   1.24033010e-02,
         8.24108899e-01], dtype=float32)

# Load Saved Attention Model
This section provides prototyping code for loading a trained attention model.

In [18]:
import os
import tensorflow as tf

## Load dataset
Load a dataset for evaluating the model.

In [19]:
from komorebi.libs.dataset.types.dataset_config import DatasetConfiguration
from komorebi.libs.dataset.types.tf_dataset_wrapper import tf_dataset_wrapper 
from komorebi.libs.trainer.trainer_utils import compute_number_of_batches, get_data_stream_for_epoch

# load dataset
dataset_config = DatasetConfiguration(
    dataset_name='validation_dataset', 
    examples_directory='/Users/andy/Projects/attention_histone_modification/datasets/attention_validation_tf_dataset')

dataset = tf_dataset_wrapper(dataset_config)

# build iterator
dataset.build_input_pipeline_iterator(batch_size=1,
                                      buffer_size=100,
                                      parallel_calls=1)

## Load Saved Model
Take a trained model and load it into memory and predict on examples.

In [23]:
saved_model_dir = "/tmp/attention_experiment_test/trained_model"

# load model
with tf.Session(graph=tf.Graph()) as sess:
    serving_tag = tf.saved_model.tag_constants.SERVING
    tf.saved_model.loader.load(sess=sess, tags=[serving_tag], export_dir=saved_model_dir)
    graph = tf.get_default_graph()
    ops = [n.name for n in graph.as_graph_def().node]
    
    

INFO:tensorflow:Restoring parameters from /tmp/attention_experiment_test/trained_model/variables/variables


INFO:tensorflow:Restoring parameters from /tmp/attention_experiment_test/trained_model/variables/variables


In [40]:
def find_substring(query, ops):
    matches = []
    for op in ops:
        if query in op:
            matches.append(op)
    return matches

# get ops
sequence_op = find_substring("sequence", ops)
feature_op = find_substring("feature", ops)
context_op = find_substring("context", ops)
logit_op = find_substring("logit", ops)
prediction_op = find_substring("prediction", ops)

In [38]:
features = graph.get_tensor_by_name("features")
sequences = graph.get_tensor_by_name("sequences")
logits = graph.get_tensor_by_name("decode_lstm/logits")
sess.run(op_to_restore,feed_dict)

[u'decode_lstm/logits',
 u'optimizer/gradients/decode_lstm/logits_grad/Shape',
 u'optimizer/gradients/decode_lstm/logits_grad/Shape_1',
 u'optimizer/gradients/decode_lstm/logits_grad/BroadcastGradientArgs',
 u'optimizer/gradients/decode_lstm/logits_grad/Sum',
 u'optimizer/gradients/decode_lstm/logits_grad/Reshape',
 u'optimizer/gradients/decode_lstm/logits_grad/Sum_1',
 u'optimizer/gradients/decode_lstm/logits_grad/Reshape_1',
 u'optimizer/gradients/decode_lstm/logits_grad/tuple/group_deps',
 u'optimizer/gradients/decode_lstm/logits_grad/tuple/control_dependency',
 u'optimizer/gradients/decode_lstm/logits_grad/tuple/control_dependency_1',
 u'decode_lstm_1/logits']

In [42]:
context_op

[u'context',
 u'decode_lstm/w_context/Initializer/random_uniform/shape',
 u'decode_lstm/w_context/Initializer/random_uniform/min',
 u'decode_lstm/w_context/Initializer/random_uniform/max',
 u'decode_lstm/w_context/Initializer/random_uniform/RandomUniform',
 u'decode_lstm/w_context/Initializer/random_uniform/sub',
 u'decode_lstm/w_context/Initializer/random_uniform/mul',
 u'decode_lstm/w_context/Initializer/random_uniform',
 u'decode_lstm/w_context',
 u'decode_lstm/w_context/Assign',
 u'decode_lstm/w_context/read',
 u'optimizer/gradients/context_grad/Shape',
 u'optimizer/gradients/context_grad/ScatterNd',
 u'decode_lstm/w_context/Adam/Initializer/zeros',
 u'decode_lstm/w_context/Adam',
 u'decode_lstm/w_context/Adam/Assign',
 u'decode_lstm/w_context/Adam/read',
 u'decode_lstm/w_context/Adam_1/Initializer/zeros',
 u'decode_lstm/w_context/Adam_1',
 u'decode_lstm/w_context/Adam_1/Assign',
 u'decode_lstm/w_context/Adam_1/read',
 u'optimizer/Adam/update_decode_lstm/w_context/ApplyAdam',
 u'co